# API Center ❤️ all APIs

## Import from Azure API Management
![image](../../images/import-from-azure-apim.png)

Playground to experiment importing APIs from Azure APIM into API Center. We start by creating APIM and API Center instances using [Bicep resource definition](https://learn.microsoft.com/en-us/azure/templates/microsoft.apicenter/services?pivots=deployment-language-bicep) and then we will use the [Azure CLI]((https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest)) to import the APIs from APIM.

▶️ Set the kernel to `.NET Interactive` before running the PowerShell scripts.

💡 Every step outlined below leverages the power of PowerShell scripts. This is designed so you can seamlessly integrate them into your automation workflows, CI/CD pipelines, webhooks, and more.

### Learning Objectives - Upon completing this lab, you should be able to:
- Deploy Azure API Management and Azure API Center using Bicep. With a role assignment allowing API Center to read APIs from APIM.
- Deploy sample APIs in APIM.
- Configure the API Center metadata scheme for APIs, Environments and Deployments.
- Understand the Azure CLI commands to manage Azure API Center. [Full list of commands available here](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest).
- Execute the [import-from-apim](https://learn.microsoft.com/en-us/cli/azure/apic/service?view=azure-cli-latest#az-apic-service-import-from-apim) command.
- List the APIs imported.

### Prerequisites
- Install or update to the latest [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download)
- Install or update to the latest [Visual Studio Code](https://code.visualstudio.com/)
- Install the [Polyglot Notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) from the VS Code marketplace
- Install or update to the latest [PowerShell](https://learn.microsoft.com/en-us/powershell/scripting/install/installing-powershell)
- Install or update to the latest [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli). The Azure API Center extension will automatically install the first time you run an [az apic](https://learn.microsoft.com/en-us/cli/azure/apic?view=azure-cli-latest) command.
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)
- [Register the Microsoft.ApiCenter provider](https://learn.microsoft.com/en-us/azure/api-center/set-up-api-center-azure-cli#register-the-microsoftapicenter-provider)

The [Oficial documentation](https://learn.microsoft.com/en-us/azure/api-center/import-api-management-apis?tabs=portal) describes in detail the process.

### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the APIC location parameter according your preferences and [region availability.](https://learn.microsoft.com/en-us/azure/api-center/overview#available-regions) 


In [ ]:

$deploymentName = Split-Path -Path (Get-Location) -Leaf
$resourceGroupName = "lab-$deploymentName" # change the name to match your naming style
$resourceGroupLocation = "westeurope"
$apicResourceNamePrefix = "apic"
$apicResourceSku = "free"
$apicResourceTags = @{
  "lab" = $deploymentName
}
$apimResourceNamePrefix = "apim"
$apimResourceLocation = "westeurope"
$apimResourceSku = "Basicv2"

# the following metadata will be created during the deployment
$metadata = @(
  @{
    "name" = 'termsOfService'
    "schema" = '{"title":"termsOfService","description":"The terms of service URL for using the API","type":"string","format":"uri"}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'externalDocs'
    "schema" = '{"title":"externalDocs","description":"External Documents that describe the API","type":"object","properties":{"description":{"title":"description","description":"Description of the External Documents","type":"string"},"url":{"title":"url","description":"The URL to reference the external documents","type":"string","format":"uri"}},"required":["url"]}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'sourceControl'
    "schema" = '{"title":"sourceControl","description":"Source Control Info","type":"object","properties":{"type":{"type":"string","title":"type","description":"Source Control System type","oneOf":[{"const":"GitHub","description":""},{"const":"GitLab","description":""},{"const":"Bitbucket","description":""},{"const":"Other","description":""}]},"repository":{"title":"repository","description":"URL for the repository","type":"string","format":"uri"},"url":{"title":"url","description":"Full URL for the API definition stored in the source control system","type":"string","format":"uri"}},"required":["type","repository","url"]}'
    "assignedTo" = @(
      @{
        "entity" = 'api'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'securitySchemes'
    "schema" = '{"title":"securitySchemes","description":"API Security Schemes","type":"array","items":{"type":"object","properties":{"name":{"title":"name","description":"Name of the API Security Scheme","type":"string"},"type":{"type":"string","title":"type","description":"The Type of the API Security Scheme","oneOf":[{"const":"http","description":"Basic, Bearer and other HTTP authentications schemes"},{"const":"apiKey","description":"API keys and cookie authentication"},{"const":"oauth2","description":"OAuth 2.0 is the industry-standard protocol for authorization."},{"const":"openIdConnect ","description":"OpenID Connect Discovery"}]},"authorizationurl":{"title":"authorizationUrl","type":"string","format":"uri"}},"required":["name","type"]}}'
    "assignedTo" = @(
      @{
        "entity" = 'deployment'
        "required" = $false
      }
    )
  },
  @{
    "name" = 'host'
    "schema" = '{"title":"host","description":"The name of the host that is hosting the API","type":"string"}'
    "assignedTo" = @(
      @{
        "entity" = 'environment'
        "required" = $false
      }
    )
  }
)

# the following environment(s) will be created during the deployment
$environments = @(
  @{
    "title" = 'prod'
    "kind" = 'production'
    "description" = 'Production Environment'
    "server" = @{
        "type" = 'Kubernetes' 
        "managementPortalUri" = 'https://swagger.io/tools/swaggerhub/'
    }
    "onboarding" = @{
        "developerPortalUri" = 'https://swagger.io/tools/swaggerhub/features/swaggerhub-portal/'
        "instructions" = 'Sign in and access the developer portal to get started with the API'
    }
    "customProperties" = @{ 
    }
  }
)

$environmentCustomProperties = @{
    "host" = "petstore3.swagger.io"
}

$apiCustomProperties = @{
    "termsOfService" = "http://swagger.io/terms/"
    "externalDocs" = @{
        "description" = "Find out more about Swagger"
        "url" = "http://swagger.io"
    }
    "sourceControl" = @{
        "type" = "GitHub"
        "repository" = "https://github.com/swagger-api/swagger-petstore"
        "url" = "https://github.com/swagger-api/swagger-petstore/blob/master/src/main/resources/openapi.yaml"
    }
}

$deploymentCustomProperties = @{
    "securitySchemes" = @(
        @{
          "name" = "petstore_auth"
          "type" = "oauth2"
          "authorizationurl" = "https://petstore3.swagger.io/oauth/authorize"
        }
    )
}

$apisConfig = @(
  @{
    "name" = 'petstore'
    "type" = 'http'
    "revision" = '1-0-20-SNAPSHOT'
    "revisionDescription" = '1.0.20-SNAPSHOT'
    "displayName" = 'Petstore'
    "description" = 'Sample Pet Store Server'
    'contact' = @{
      'name' = 'API Team'
      'email' = 'apiteam@swagger.io'
      'url' = 'http://swagger.io'
    }
    'license' = @{
      'name' = 'Apache 2.0'
      'url' = 'http://www.apache.org/licenses/LICENSE-2.0.html'
    }
    'format' = 'openapi-link'
    "path" = 'petstore'
    'termsOfServiceUrl' = 'http://swagger.io/terms/'
    "serviceUrl" = 'https://petstore3.swagger.io/api/v3'
    "specURL" = 'https://raw.githubusercontent.com/swagger-api/swagger-petstore/master/src/main/resources/openapi.yaml'
  },
  @{
    "name" = 'openai'
    "type" = 'http'
    "revision" = '2024-02-01'
    "revisionDescription" = '2024-02-01'
    "displayName" = 'OpenAI'
    "description" = 'OpenAI inference API'
    'contact' = @{
      'name' = 'OpenAI Team'
      'email' = 'openai@microsoft.com'
      'url' = 'http://www.microsoft.com'
    }
    'license' = @{
      'name' = 'MIT License'
      'url' = 'Http://opensource.org/licenses/MIT'
    }
    'format' = 'openapi-link'
    "path" = 'openai'
    'termsOfServiceUrl' = 'https://learn.microsoft.com/en-us/legal/cognitive-services/openai/code-of-conduct'
    "serviceUrl" = 'https://your-resource-name.openai.azure.com'
    "specURL" = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/2024-02-01/inference.json'
  }
)

Write-Output "✅ Variables initialized ⌚ $(Get-Date -Format 'HH:mm:ss')"


### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
$resourceGroupOutput = az group create --name $resourceGroupName --location $resourceGroupLocation

if ($LASTEXITCODE -ne 0) {
    Write-Output $resourceGroupOutput
} else {
    Write-Output "✅ Azure Resource Grpup $resourceGroupName created ⌚ $(Get-Date -Format 'HH:mm:ss')"
}

### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
$bicepParameters = @{
    "`$schema" = "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#"
    "contentVersion" = "1.0.0.0"
    "parameters" = @{
        "apicResourceNamePrefix" = @{ "value" = $apicResourceNamePrefix }
        "apicResourceTags" = @{ "value" = $apicResourceTags }
        "apicMetadataSchema" = @{ "value" = $metadata }
        "apicEnvironments" = @{ "value" = $environments }
        "apimResourceNamePrefix" = @{ "value" = $apimResourceNamePrefix }
        "apimResourceLocation" = @{ "value" = $apimResourceLocation }
        "apimResourceSku" = @{ "value" = $apimResourceSku }
        "apisConfig" = @{ "value" = $apisConfig }
    }
}

$bicepParametersJson = ConvertTo-Json -InputObject $bicepParameters -Depth 10
Set-Content -Path "params.json" -Value $bicepParametersJson

# Execute the Azure CLI command to create the deployment
az deployment group create --name $deploymentName --resource-group $resourceGroupName --template-file "main.bicep" --parameters "params.json"



### 3️⃣ Get the deployment outputs

We will set the `apicResourceName` variable with the value that was returned from the deployment 

In [ ]:
$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apicResourceName.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apicResourceName = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Center name: $apicResourceName"

$deploymentOutput = az deployment group show --name $deploymentName -g $resourceGroupName --query properties.outputs.apimResourceId.value -o json
if ($LASTEXITCODE -ne 0) {
    Write-Output $deploymentOutput
} else {
    $apimResourceId = $deploymentOutput | ConvertFrom-Json
}
Write-Output "👉🏻 API Management id: $apimResourceId"

### 4️⃣ Update the environment metadata

We will [update the enviroment](https://learn.microsoft.com/en-us/cli/azure/apic/environment?view=azure-cli-latest#az-apic-environment-update) with the `custom properties` defined in the variable `environmentCustomProperties`

In [ ]:
$environmentCustomPropertiesJson = ConvertTo-Json -InputObject $environmentCustomProperties -Depth 10 -Compress
az apic environment update -g $resourceGroupName -s $apicResourceName --environment-id "prod" `
            --custom-properties $environmentCustomPropertiesJson.replace('"','\"')

### 5️⃣ Import the Azure API Management APIs into Azure API Center

We will use the [import-from-apim](https://learn.microsoft.com/en-us/cli/azure/apic/service?view=azure-cli-latest#az-apic-service-import-from-apim) command with the wildcard to import all the APIs.

In [ ]:
az apic service import-from-apim -g $resourceGroupName -s $apicResourceName --source-resource-ids "$apimResourceId/apis/*"

### 6️⃣ Discover the APIs that were just imported

You can discover the APIs with fhe following methods:
- With the Azure Portal
- With the [self-hosted API Center Portal](https://learn.microsoft.com/en-us/azure/api-center/enable-api-center-portal)
- With the [VS Code extension](https://learn.microsoft.com/en-us/azure/api-center/use-vscode-extension-copilot) that is integrated with GitHub Copilot Chat.
- With the CLI, the service REST API and more

Here we will use the [list command](https://learn.microsoft.com/en-us/cli/azure/apic/api?view=azure-cli-latest#az-apic-api-list) to display the APIs that we have just imported. 


In [ ]:
az apic api list -g $resourceGroupName -s $apicResourceName --query "[].{Name:name, Title:title, Kind:kind, ContantEmail:contacts[0].email}" -o table

### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.